In [ ]:
import momepy
import folium
import osmnx as ox
import graphbuilder_v2

In [ ]:
city = ox.geocode_to_gdf('R1281563', by_osmid=True)
graph = graphbuilder_v2.get_graph_polygon(city.unary_union, crs=32636)

In [ ]:
graph = graphbuilder_v2.convert_geometry_to_wkt(graph)

In [ ]:
nodes, edges = momepy.nx_to_gdf(graph, points=True, lines=True, spatial_weights=False)

In [ ]:
edges = edges.to_crs(epsg=4326)
edges_filtered = edges[edges['reg'].isin([1, 2])]

m = folium.Map()

# Определение цветов для разных значений reg
color_map = {1: 'red', 2: 'blue', 3: 'green'}

# Добавление рёбер на карту
for _, row in edges_filtered.iterrows():
    folium.GeoJson(
        row['geometry'],
        style_function=lambda feature, color=color_map[row['reg']]: {'color': color},
        tooltip=f"highway: {row['highway']}, ref: {row['ref']}, reg: {row['reg']}"
    ).add_to(m)

# Отображение карты
m